In [2]:
import gym
import __init__
import time
import numpy as np
#0: up
#1: down
#2: left
#3: right
#4: nothing

#enter a number to select agent and move it with wasd

env = gym.make('pressureplate-custom-4p-v0')
state = env.reset()
print(state[0])
env.render()
agent = 0
for _ in range(1000):
    action = [4,4,4,4]
    _input = input("Enter action as wasd or number to change agent:")

    if _input in ["0","1","2","3"]:
        agent = int(_input)
        continue
    elif _input in ["w", "a", "s", "d"]:
        if _input == "w":
            action[agent] = 0
        elif _input == "a":
            action[agent] = 2
        elif _input == "d":
            action[agent] = 3
        else:
            action[agent] = 1
    else:
        continue

    next_state, reward, dontknow, _ =  env.step(action)
    print("agent 1:")
    print(next_state[1])
    print("agent 3:")
    print(next_state[3])
    env.render()

/home/mert/miniconda3/envs/pressureplate/lib/python3.11/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/mert/miniconda3/envs/pressureplate/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/mert/miniconda3/envs/pressureplate/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/home/mert/miniconda3/envs/pressureplate/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:199: UserWarning: WARN: The result returned by `env.reset()` should be 

[ 1.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  2.  2.  2.  2.  2. 26.  7.  4.]


KeyboardInterrupt: Interrupted by user

In [1]:
import gym
import __init__
import time
import numpy as np
from MADDPGV2.maddpg import MADDPG
from MADDPGV2 import gradient_estimators
from MADDPGV2.buffer import ReplayBuffer
from time import sleep, time

env = gym.make('pressureplate-custom-4p-v0')
obs_dim = env.observation_space
obs_dim = 2

render = True
max_episode_length = 30
max_time_steps = 10000

gradient_estimator = gradient_estimators.TAGS(10, 0.5, max_time_steps)

maddpg = MADDPG(
        env=env,
        critic_lr=3e-4,
        actor_lr=3e-4,
        gradient_clip=1.0,
        hidden_dim_width=64,
        gamma=0.95,
        soft_update_size=0.01,
        policy_regulariser=0.001,
        gradient_estimator= gradient_estimator,
        standardise_rewards=False,
        pretrained_agents=""
    )

observation_dims = np.array([obs.shape[0] for obs in env.observation_space])
buffer = ReplayBuffer(capacity=2_000_000, obs_dims= observation_dims, batch_size = 256)

obs = env.reset()
dones = [False] * env.n_agents

episode_steps = 0
episode_return = np.zeros((env.n_agents,))

for i in range(max_time_steps):
    print("time step:", i)
    if render:
        if i > 9000:
            env.render()
            sleep(0.03)
    
    if all(dones):
        obs = env.reset()
        dones = [False] * env.n_agents
        episode_steps = 0
        episode_return = np.zeros((env.n_agents,))
    if i < 2000:
         acts = env.action_space.sample()
    else:
         acts = maddpg.acts(obs)
    nobs, rwds, dones, _ = env.step(acts)
    episode_steps += 1

    if (episode_steps >= max_episode_length): # Some envs don't have done flags,
        dones = [True] * env.n_agents

    if buffer is not None:
                buffer.store(
                    obs=obs,
                    acts=acts,
                    rwds=rwds,
                    nobs=nobs,
                    dones=dones,
                )
    episode_return += np.array(rwds)
    print(rwds)
    obs = nobs

    sample = buffer.sample()
    if sample is not None:
        maddpg.update(sample)

/home/mert/miniconda3/envs/pressureplate/lib/python3.11/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment pressureplate-linear-1p-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/mert/miniconda3/envs/pressureplate/lib/python3.11/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment pressureplate-custom-4p-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/mert/miniconda3/envs/pressureplate/lib/python3.11/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment pressureplate-linear-4p-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/mert/miniconda3/envs/pressureplate/lib/python3.11/site-packages/gym/envs/registration.py:498: UserWarning: WARN: Overriding environment pressureplate-linear-5p-v0 already in registry.
  logger.war

time step: 0
[-0.8, -0.4, -0.4, -1.0]
time step: 1
[-0.8, -0.4, -0.5, -1.0]
time step: 2
[-0.7, -0.3, -0.4, -0.9]
time step: 3
[-0.7, -0.2, -0.5, -1.0]
time step: 4
[-0.6, -0.1, -0.6, -1.0]
time step: 5
[-0.5, 10, -0.7, -1.1]
time step: 6
[-0.4, 10, -0.7, -1.0]
time step: 7
[-0.3, -0.1, -0.7, -1.0]
time step: 8
[-0.3, -0.2, -0.7, -1.0]
time step: 9
[-0.3, -0.3, -0.8, -0.9]
time step: 10
[-0.2, -0.3, -0.7, -1.0]
time step: 11
[-0.2, -0.2, -0.7, -0.9]
time step: 12
[-0.3, -0.1, -0.6, -0.9]
time step: 13
[-0.3, 10, -0.5, -0.8]
time step: 14
[-0.4, 10, -0.4, -0.8]
time step: 15
[-0.3, -0.1, -0.3, -0.8]
time step: 16
[-0.4, -0.1, -0.4, -0.9]
time step: 17
[-0.3, -0.1, -0.4, -1.0]
time step: 18
[-0.3, -0.1, -0.3, -1.1]
time step: 19
[-0.3, 10, -0.3, -1.1]
time step: 20
[-0.3, 10, -0.3, -1.0]
time step: 21
[-0.3, 10, -0.3, -1.1]
time step: 22
[-0.2, 10, -0.3, -1.0]
time step: 23
[-0.1, 10, -0.3, -1.0]
time step: 24
[-0.1, 10, -0.2, -1.1]
time step: 25
[-0.2, -0.1, -0.2, -1.1]
time step: 26
[-

/home/mert/miniconda3/envs/pressureplate/lib/python3.11/site-packages/torch/autograd/__init__.py:197: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


time step: 256
[-0.6, -0.4, -0.5, -0.8]
time step: 257
[-0.7, -0.5, -0.6, -0.8]
time step: 258
[-0.8, -0.6, -0.6, -0.7]
time step: 259
[-0.9, -0.6, -0.7, -0.7]
time step: 260
[-0.8, -0.5, -0.6, -0.6]
time step: 261
[-0.9, -0.6, -0.7, -0.5]
time step: 262
[-1.0, -0.5, -0.8, -0.6]
time step: 263
[-1.1, -0.6, -0.7, -0.5]
time step: 264
[-1.0, -0.5, -0.8, -0.6]
time step: 265
[-1.0, -0.5, -0.8, -0.7]
time step: 266
[-0.9, -0.6, -0.8, -0.6]
time step: 267
[-0.8, -0.7, -0.8, -0.7]
time step: 268
[-0.7, -0.6, -0.8, -0.6]
time step: 269
[-0.7, -0.6, -0.7, -0.5]
time step: 270
[-1.0, -0.3, -0.4, -1.0]
time step: 271
[-0.9, -0.3, -0.4, -1.1]
time step: 272
[-1.0, -0.2, -0.5, -1.2]
time step: 273
[-1.1, -0.3, -0.4, -1.1]
time step: 274
[-1.0, -0.4, -0.5, -1.2]
time step: 275
[-1.1, -0.5, -0.4, -1.1]
time step: 276
[-1.2, -0.6, -0.4, -1.0]
time step: 277
[-1.2, -0.7, -0.5, -1.0]
time step: 278
[-1.2, -0.7, -0.6, -1.0]
time step: 279
[-1.1, -0.6, -0.5, -1.0]
time step: 280
[-1.0, -0.7, -0.6, -0.9]


/home/mert/miniconda3/envs/pressureplate/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:272: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(


[-0.7, -0.2, -0.2, -0.9]
time step: 9002
[-0.6, -0.1, -0.1, -0.8]
time step: 9003
[-0.5, 10, 10, -0.7]
time step: 9004
[-0.4, 10, 10, -0.7]
time step: 9005
[-0.3, 10, 10, -0.7]
time step: 9006
[-0.2, 10, 10, -0.6]
time step: 9007
[-0.1, 10, 10, -0.5]
time step: 9008
[10, 10, 10, -0.4]
time step: 9009
[10, 10, 10, -0.3]
time step: 9010
[10, 10, 10, -0.3]
time step: 9011
[10, 10, 10, -0.3]
time step: 9012
[10, 10, 10, -0.3]
time step: 9013
[10, 10, 10, -0.3]
time step: 9014
[10, 10, 10, -0.3]
time step: 9015
[10, 10, 10, -0.3]
time step: 9016
[10, 10, 10, -0.3]
time step: 9017
[10, 10, 10, -0.3]
time step: 9018
[10, 10, 10, -0.3]
time step: 9019
[10, 10, 10, -0.3]
time step: 9020
[10, 10, 10, -0.3]
time step: 9021
[10, 10, 10, -0.3]
time step: 9022
[10, 10, 10, -0.3]
time step: 9023
[10, 10, 10, -0.3]
time step: 9024
[10, 10, 10, -0.3]
time step: 9025
[10, 10, 10, -0.3]
time step: 9026
[10, 10, 10, -0.3]
time step: 9027
[10, 10, 10, -0.3]
time step: 9028
[10, 10, 10, -0.3]
time step: 902

In [4]:
s = "abc"
s[5]

IndexError: string index out of range

In [2]:
import gym
import __init__
import time
import numpy as np
from MADDPG.buffer import MultiAgentReplayBuffer
from MADDPG.maddpg import MADDPG

env = gym.make('pressureplate-custom-4p-v0')
state = env.reset()
obs_dim = env.observation_space[0].shape[0]
n_agents = env.n_agents
obs_dim = 2



replay_buffer = MultiAgentReplayBuffer(max_size = 10000, single_state_dim = obs_dim , n_agents = 1, batch_size = 10)
model = MADDPG(actor_dims = obs_dim, critic_dims = obs_dim * n_agents + n_agents * 5, n_actions = 5, n_agents = 1, gamma = 0)
epsilon = 0.6

for i in range(10000):
    encoded_action = [0,0,0,0,0]
    
    if i % 500 == 0:
        state = env.reset()
    if i >9000:
        epsilon = 0.8
    if i < 2000:
        action = env.action_space.sample()
    else:
        action = model.choose_action(state, epsilon = epsilon)
    next_state, reward, done , _ = env.step(action)


    print("reward:", reward)
    print("action:", action)

    encoded_action[action[0]] = 1
    replay_buffer.add(state, next_state, encoded_action, reward, done)
    state = next_state

    if i > 100:
        model.learn(replay_buffer)
    






reward: [-1.0]
action: (1,)
reward: [-1.1]
action: (2,)
reward: [-1.1]
action: (1,)
reward: [-1.0]
action: (3,)
reward: [-1.1]
action: (2,)
reward: [-1.1]
action: (1,)
reward: [-1.0]
action: (0,)
reward: [-1.1]
action: (2,)
reward: [-1.2]
action: (2,)
reward: [-1.2]
action: (4,)
reward: [-1.2]
action: (4,)
reward: [-1.3]
action: (1,)
reward: [-1.3]
action: (1,)
reward: [-1.3]
action: (1,)
reward: [-1.3]
action: (1,)
reward: [-1.3]
action: (4,)
reward: [-1.3]
action: (1,)
reward: [-1.3]
action: (2,)
reward: [-1.2]
action: (0,)
reward: [-1.2]
action: (2,)
reward: [-1.1]
action: (3,)
reward: [-1.0]
action: (0,)
reward: [-0.9]
action: (0,)
reward: [-0.8]
action: (0,)
reward: [-0.7]
action: (3,)
reward: [-0.6]
action: (3,)
reward: [-0.7]
action: (1,)
reward: [-0.6]
action: (3,)
reward: [-0.5]
action: (3,)
reward: [-0.5]
action: (4,)
reward: [-0.5]
action: (4,)
reward: [-0.6]
action: (1,)
reward: [-0.5]
action: (0,)
reward: [-0.4]
action: (3,)
reward: [-0.5]
action: (1,)
reward: [-0.6]
actio

/home/mert/miniconda3/envs/pressureplate/lib/python3.11/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/mert/miniconda3/envs/pressureplate/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/mert/miniconda3/envs/pressureplate/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/home/mert/miniconda3/envs/pressureplate/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:199: UserWarning: WARN: The result returned by `env.reset()` should be 

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x3 and 7x128)

In [34]:
import torch as T
coors = []
row = []
for x in range(4,14):
    row = []
    for y in range(23,28):
        row.append([x,y])
    coors.append(row)
coors = np.array(coors)
coors = coors.reshape(-1,2)

coors2 = []
row = []
for y in range(23,28):
    row = []
    for x in range(4,14):
        for a in range(0,5):
            row.append([x,y,a])
    coors2.append(row)
coors2 = np.array(coors2)
coors2 = coors2.reshape(-1,3)
coors2.shape
print(coors2)

[[ 4 23  0]
 [ 4 23  1]
 [ 4 23  2]
 [ 4 23  3]
 [ 4 23  4]
 [ 5 23  0]
 [ 5 23  1]
 [ 5 23  2]
 [ 5 23  3]
 [ 5 23  4]
 [ 6 23  0]
 [ 6 23  1]
 [ 6 23  2]
 [ 6 23  3]
 [ 6 23  4]
 [ 7 23  0]
 [ 7 23  1]
 [ 7 23  2]
 [ 7 23  3]
 [ 7 23  4]
 [ 8 23  0]
 [ 8 23  1]
 [ 8 23  2]
 [ 8 23  3]
 [ 8 23  4]
 [ 9 23  0]
 [ 9 23  1]
 [ 9 23  2]
 [ 9 23  3]
 [ 9 23  4]
 [10 23  0]
 [10 23  1]
 [10 23  2]
 [10 23  3]
 [10 23  4]
 [11 23  0]
 [11 23  1]
 [11 23  2]
 [11 23  3]
 [11 23  4]
 [12 23  0]
 [12 23  1]
 [12 23  2]
 [12 23  3]
 [12 23  4]
 [13 23  0]
 [13 23  1]
 [13 23  2]
 [13 23  3]
 [13 23  4]
 [ 4 24  0]
 [ 4 24  1]
 [ 4 24  2]
 [ 4 24  3]
 [ 4 24  4]
 [ 5 24  0]
 [ 5 24  1]
 [ 5 24  2]
 [ 5 24  3]
 [ 5 24  4]
 [ 6 24  0]
 [ 6 24  1]
 [ 6 24  2]
 [ 6 24  3]
 [ 6 24  4]
 [ 7 24  0]
 [ 7 24  1]
 [ 7 24  2]
 [ 7 24  3]
 [ 7 24  4]
 [ 8 24  0]
 [ 8 24  1]
 [ 8 24  2]
 [ 8 24  3]
 [ 8 24  4]
 [ 9 24  0]
 [ 9 24  1]
 [ 9 24  2]
 [ 9 24  3]
 [ 9 24  4]
 [10 24  0]
 [10 24  1]
 [10 24  2]
 [10

In [35]:
#0: up
#1: down
#2: left
#3: right
#4: nothing
env.render()
critic_input = T.tensor(coors2, dtype = T.float)
print(T.argmax(model.agents[0].critic(critic_input).flatten().view(5,10,5), dim = -1))
print(model.agents[0].critic(critic_input).flatten().view(5,10,5))
actor_input = T.tensor(coors, dtype = T.float)

print(model.agents[0].actor(actor_input).flatten().view(5,10))

tensor([[4, 4, 4, 4, 4, 4, 4, 4, 0, 0],
        [4, 0, 4, 4, 4, 4, 4, 4, 4, 0],
        [4, 0, 0, 4, 4, 4, 4, 4, 4, 0],
        [4, 0, 0, 0, 0, 0, 4, 4, 4, 0],
        [4, 0, 0, 0, 0, 0, 0, 4, 4, 4]])
tensor([[[-1.0551, -1.1621, -1.1189, -1.0673, -1.0308],
         [-0.9639, -1.0687, -1.0283, -0.9908, -0.9261],
         [-0.8834, -0.9831, -0.9525, -0.9408, -0.8144],
         [-0.7348, -0.8771, -0.9099, -0.8982, -0.6707],
         [-0.5925, -0.7703, -0.8168, -0.7773, -0.4749],
         [-0.5715, -0.7669, -0.7766, -0.6627, -0.3199],
         [-0.4861, -0.7215, -0.7362, -0.4611, -0.0305],
         [-0.0330, -0.3989, -0.3893,  0.1462,  0.7423],
         [ 3.4162,  2.2102,  1.7230,  1.9585,  2.2632],
         [ 8.2156,  6.9288,  5.9864,  5.6250,  5.3194]],

        [[-1.1504, -1.2575, -1.2251, -1.1735, -1.1195],
         [-1.0570, -1.1640, -1.1345, -1.0829, -1.0602],
         [-0.9761, -1.0758, -1.0455, -1.0326, -0.9485],
         [-0.8738, -0.9952, -1.0092, -0.9890, -0.8275],
         [-0.

/home/mert/miniconda3/envs/pressureplate/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:272: UserWarning: WARN: No render modes was declared in the environment (env.metadata['render_modes'] is None or not defined), you may have trouble when calling `.render()`.
  logger.warn(


In [21]:
for a in range(5):
    critic_input = T.tensor([[ 6, 23,  a]], dtype = T.float)
    print(model.agents[0].critic(critic_input).flatten())

tensor([-31.3011], grad_fn=<ViewBackward0>)
tensor([-31.8005], grad_fn=<ViewBackward0>)
tensor([-31.5978], grad_fn=<ViewBackward0>)
tensor([-30.9491], grad_fn=<ViewBackward0>)
tensor([-30.2241], grad_fn=<ViewBackward0>)


In [2]:
env = gym.make('pressureplate-custom-4p-v0')
state = env.reset()
for i in range(1000):
    action = model.choose_action(state, epsilon = 1)
    next_state, reward, done , _ = env.step(action)
    state = next_state
    env.render()
    print("reward:", reward)
    print("action:", action)

    

/home/mert/miniconda3/envs/pressureplate/lib/python3.11/site-packages/gym/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/home/mert/miniconda3/envs/pressureplate/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/home/mert/miniconda3/envs/pressureplate/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:187: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/home/mert/miniconda3/envs/pressureplate/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:199: UserWarning: WARN: The result returned by `env.reset()` should be 

reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
reward: [-1.0]
action: [1]
r

KeyboardInterrupt: 

In [4]:
import gym
import __init__
import time
import numpy as np
from MADDPG.buffer import MultiAgentReplayBuffer
from MADDPG.maddpg import MADDPG

env = gym.make('pressureplate-custom-4p-v0')
state = env.reset()
obs_dim = env.observation_space[0].shape[0]
n_agents = env.n_agents
obs_dim = 2



replay_buffer = MultiAgentReplayBuffer(max_size = 10000, single_state_dim = obs_dim , n_agents = 1, batch_size = 11)
model = MADDPG(actor_dims = obs_dim, critic_dims = obs_dim * n_agents + n_agents, n_actions = 5, n_agents = 1, gamma = 0.99)


for i in range(11):
    action = model.choose_action(state)
    next_state, reward, done , _ = env.step(action)    
    replay_buffer.add(state, next_state, action, reward, done)
    state = next_state

In [5]:
model.learn(replay_buffer)

states tensor: torch.Size([11, 2])
actions tensor: torch.Size([11, 1])
rewards tensor: torch.Size([11, 1])
states_ tensor: torch.Size([11, 2])
terminal tensor: torch.Size([11, 1])
target_actor_out: torch.Size([11, 1])
actor_out: torch.Size([11, 1])
target out after contac: torch.Size([11, 1])
target out after contac: torch.Size([11, 1])


In [ ]:
arr = np.arange(16)
np.random.shuffle(arr)
arr = arr.reshape((4,4))
arr = T.tensor(arr, dtype = T.float)
arr = T.softmax(arr, dim = 1)
arr

tensor([[8.3076e-07, 9.9907e-01, 1.6686e-05, 9.1104e-04],
        [7.9208e-07, 4.7425e-02, 9.5256e-01, 1.5909e-05],
        [2.4398e-04, 4.9005e-03, 2.6756e-01, 7.2730e-01],
        [7.9781e-04, 8.7490e-01, 1.1841e-01, 5.8950e-03]])

In [ ]:
T.argmax(arr, dim = 1, keepdim= True)

tensor([[1],
        [2],
        [3],
        [1]])